In [ ]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name="model"):
        self.state_size = state_size # normatlized previous days
        self.action_size = 5
        self.memory = deque(maxlen=int(len(initial_data)))
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.model = load_model(r"C:...\model") if is_eval else self._model()
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="act"))
        model.add(Dense(units=32, activation="act"))
        model.add(Dense(units=5, activation="act"))
        
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random()<= self.epsilon:
            return random.randrange(self.action_size)
        
        options = self.model.predict(next_state)
        print(options[0],np.argmax(options[0]),"options") 
        return np.argmax(options[0])
        
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            print(self.model.predict(next_state)[0],"prednext",target,"target",target_f[0],"targetf")
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay